In [10]:
from pytrends.request import TrendReq
import datetime

# Login to Google. Only need to run this once, the rest of requests will use the same session.
pytrend = TrendReq()

In [11]:
def getWeeks():

    dStart = input("Enter date of start epidemiological week in DD-MM-YYYY format: ")
    dStart = [int(i) for i in dStart.split("-")]
    start = datetime.datetime(dStart[2],dStart[1],dStart[0])
    
    dEnd = input("Enter date of end epidemiological week in DD-MM-YYYY format: ")
    dEnd = [int(i) for i in dEnd.split("-")]
    end = datetime.datetime(dEnd[2],dEnd[1],dEnd[0])
    
    nextWeek = getNextWeek(start)
    weeks = [start]
    
    while nextWeek <= end:
        if(nextWeek < datetime.datetime.today()):
            weeks.append(nextWeek)
            nextWeek = getNextWeek(nextWeek)
    
    return weeks

In [12]:
def getNextWeek(timeframe):
    return timeframe+datetime.timedelta(days=7)

In [13]:
def getYMD(timeframe):
    return "{}/{}/{}".format(timeframe.day,timeframe.month,timeframe.year)

In [14]:
year = input("Enter year: ")

Enter year: 2017


In [15]:
countryCode = input("Enter country code: ")

Enter country code: BR


In [16]:
weeks = getWeeks()

Enter date of start epidemiological week in DD-MM-YYYY format: 02-01-2017
Enter date of end epidemiological week in DD-MM-YYYY format: 31-12-2017


In [17]:
with open("{}/google/{}.csv".format(year, countryCode), "w") as file:
    weekEpiArray = []
    cities = {}
    file.write("CITY,")
    file.write(",".join(map(getYMD, weeks))+"\n")
    for i in range(len(weeks)):
        week = weeks[i]
        nextWeek = getNextWeek(week)
        year = week.year
        month = week.month
        day = week.day
        nextWeekC = nextWeek - datetime.timedelta(days=1)
        timeframe = "{}-{}-{} {}-{}-{}".format(year,month,day,nextWeekC.year,nextWeekC.month,nextWeekC.day)
        print("{}-{}-{} -> {}-{}-{}".format(year,month,day,nextWeekC.year,nextWeekC.month,nextWeekC.day))
        pytrend.build_payload(kw_list=['zika'], geo=countryCode, timeframe=timeframe)
        results = pytrend.interest_by_region(resolution="CITY")
        # if a city's search index is 0 it doesn't appear on results, so we have to keep track of it
        tempWeekArray = {}
        for index, row in results.iterrows():
            tempWeekArray[index] = row["zika"]
            if index not in cities:
                cities[index] = index #Store in dictionary to get O(1) access
        weekEpiArray.append(tempWeekArray)
        print("DONE", week)
        
    # Iterate on weekEpiArray and replace non existent cities with zeroes
    searchesByCity = {}
    for city in cities:
        searchesByCity[city] = []
        for week in weekEpiArray:
            if city not in week:
                searchesByCity[city].append(0)
            else:
                searchesByCity[city].append(week[city])
                
    # Finally write to file
    for city in sorted(searchesByCity.keys()):
        file.write("{},".format(city))
        file.write(",".join(map(str,searchesByCity[city])))
        file.write("\n")

2017-1-2 -> 2017-1-8
DONE 2017-01-02 00:00:00
2017-1-9 -> 2017-1-15
DONE 2017-01-09 00:00:00
2017-1-16 -> 2017-1-22
DONE 2017-01-16 00:00:00
2017-1-23 -> 2017-1-29
DONE 2017-01-23 00:00:00
2017-1-30 -> 2017-2-5
DONE 2017-01-30 00:00:00
2017-2-6 -> 2017-2-12
DONE 2017-02-06 00:00:00
2017-2-13 -> 2017-2-19
DONE 2017-02-13 00:00:00
2017-2-20 -> 2017-2-26
DONE 2017-02-20 00:00:00
2017-2-27 -> 2017-3-5
DONE 2017-02-27 00:00:00
2017-3-6 -> 2017-3-12
DONE 2017-03-06 00:00:00
2017-3-13 -> 2017-3-19
DONE 2017-03-13 00:00:00
2017-3-20 -> 2017-3-26
DONE 2017-03-20 00:00:00
2017-3-27 -> 2017-4-2
DONE 2017-03-27 00:00:00
2017-4-3 -> 2017-4-9
DONE 2017-04-03 00:00:00
2017-4-10 -> 2017-4-16
DONE 2017-04-10 00:00:00
2017-4-17 -> 2017-4-23
DONE 2017-04-17 00:00:00
2017-4-24 -> 2017-4-30
DONE 2017-04-24 00:00:00
2017-5-1 -> 2017-5-7
DONE 2017-05-01 00:00:00
2017-5-8 -> 2017-5-14
DONE 2017-05-08 00:00:00
2017-5-15 -> 2017-5-21
DONE 2017-05-15 00:00:00
2017-5-22 -> 2017-5-28
DONE 2017-05-22 00:00:00
2017-